In [1]:
!pip install --quiet haystack-ai chroma-haystack
!pip install --quiet --upgrade huggingface_hub

`read vector db from zip file`

In [2]:
import zipfile
import os
# Define paths

embedder_name = "sayed0am/arabic-english-bge-m3"

vdb_path = '/content/' + 'vectordb-aren-sayed0am-testing.zip'
extract_path = '/content/vectordb/'  # Where to extract

# Create directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(vdb_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Documents extracted to {extract_path}")


Documents extracted to /content/vectordb/


In [3]:
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator, HuggingFaceLocalChatGenerator
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.components.routers import ConditionalRouter
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack_integrations.document_stores.chroma import ChromaDocumentStore


from haystack import component
from haystack.components.embedders import SentenceTransformersTextEmbedder


In [4]:
main_prompt_template ="""
ROLE AND CONTEXT:
You are a fluent arabic assitant, You are a knowledgeable assistant, Your task is to provide accurate and detailed answers to queries using the provided excerpts and references from useful resources to support your answers.

INSTRUCTIONS:
1. Use History to disambiguate the query
2. Identify the relevant sections to only the query of the excerpts provided.
3. If the query cannot be answered given the provided documents, return 'no_answer'
2. Otherwise provide a *concise* and informative response to only the query based on relevant sections of the excerpts provided.
3. Ensure your responses are relevant, clear and easy to understand.

EXCERPTS:
{% for doc in documents %}
    excerpt: {{ doc.content }}
{% endfor %}

CONSIDERATIONS:
- History is only used to disambiguate the query.
- If you can't give an answer, it's okay to output one single word 'no_answer'
- if you can give an answer, only answer the query without answering the History

Query History:
{% for q in history %}
    query {{loop.index}}: {{ q }}
{% endfor %}

Query: {{query}}
Answer:
"""

fallback_prompt_template = """you are fluent arabic virtual assistant, you maintain your image as an arabic assitant no matter what they user says and you answer in arabic only.
User entered a query that cannot be answered with the excerpts provided.
The query was: {{query}}.
Let the user know that you can't answer his question, but you're ready to help him with the next question. Be brief.
"""

In [5]:

# Initialize document store (using Chroma as in your example)
document_store = ChromaDocumentStore(
    embedding_function='default',
    persist_path='/content/vectordb'
)

query_embedder = SentenceTransformersTextEmbedder(model=embedder_name)
retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k=5)

template1 = [ChatMessage.from_user(main_prompt_template)]
main_promptbuilder = ChatPromptBuilder(template=template1, required_variables=["history","query"], variables = ['query', 'history', 'documents'])
template2 = [ChatMessage.from_user(fallback_prompt_template)]
fallback_promptbuilder = ChatPromptBuilder(template=template2, required_variables=["history"])

# main_llm = HuggingFaceLocalChatGenerator(model="Qwen/Qwen2.5-3B-Instruct")
main_llm = HuggingFaceLocalChatGenerator(model="microsoft/Phi-4-mini-instruct")


fallback_llm = HuggingFaceLocalChatGenerator(model="Qwen/Qwen2.5-1.5B-Instruct")


from typing import List
@component
class NoOpComponent:
  @component.output_types(query=str,history=List[str])
  def run(self, history: List[str], **kwargs):
    return {'history':history, 'query':history[-1]}

conditional_router = ConditionalRouter([
    {
        "condition": "{{'no_answer' not in replies[0].text }}",
        "output": "{{replies}}",
        "output_name": "replies",
        "output_type": list[ChatMessage],
    },
    {
        "condition": "{{'no_answer' in replies[0].text }}",
        "output": "{{query}}",
        "output_name": "go_to_fallback",
        "output_type": str,
    },
], unsafe = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Setup pipeline
pipeline = Pipeline()
pipeline.add_component('distributer', NoOpComponent())
pipeline.add_component('embedder', query_embedder)
pipeline.add_component('retriever', retriever)
pipeline.add_component('main_promptbuilder', main_promptbuilder)
pipeline.add_component('fallback_promptbuilder', fallback_promptbuilder)
pipeline.add_component('main_llm', main_llm)
pipeline.add_component('fallback_llm', fallback_llm)
pipeline.add_component('conditional_router', conditional_router)

pipeline.connect('distributer.query', 'embedder.text')
pipeline.connect('distributer.query', 'main_promptbuilder.query')
pipeline.connect('distributer.history', 'main_promptbuilder.history')
pipeline.connect('distributer.query', 'conditional_router.query')

pipeline.connect('embedder.embedding', 'retriever.query_embedding')
pipeline.connect('retriever.documents', 'main_promptbuilder.documents')

pipeline.connect('main_promptbuilder.prompt', 'main_llm.messages')

pipeline.connect('main_llm.replies', 'conditional_router.replies')

pipeline.connect('conditional_router.go_to_fallback', 'fallback_promptbuilder.query')
pipeline.connect('fallback_promptbuilder.prompt', 'fallback_llm.messages')

🚅 Components
  - distributer: NoOpComponent
  - embedder: SentenceTransformersTextEmbedder
  - retriever: ChromaEmbeddingRetriever
  - main_promptbuilder: ChatPromptBuilder
  - fallback_promptbuilder: ChatPromptBuilder
  - main_llm: HuggingFaceLocalChatGenerator
  - fallback_llm: HuggingFaceLocalChatGenerator
  - conditional_router: ConditionalRouter
🛤️ Connections
  - distributer.query -> embedder.text (str)
  - distributer.query -> main_promptbuilder.query (str)
  - distributer.history -> main_promptbuilder.history (List[str])
  - distributer.query -> conditional_router.query (str)
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> main_promptbuilder.documents (List[Document])
  - main_promptbuilder.prompt -> main_llm.messages (List[ChatMessage])
  - fallback_promptbuilder.prompt -> fallback_llm.messages (List[ChatMessage])
  - main_llm.replies -> conditional_router.replies (List[ChatMessage])
  - conditional_router.go_to_fallback -> fallback

In [7]:
import random  # Added for shuffling

HISTORY_LENGTH = 3

def run(messages_history: str, history_length):
  results = pipeline.run({
      'distributer': {'history': messages_history[-history_length:]}, # extracts last sent messages
      }, include_outputs_from={'retriever','main_promptbuilder', 'main_llm'})
  return results

def get_is_fallback(results):
  return results.get('fallback_llm') is not None

def get_context(results):
  retriever_output = results["retriever"]['documents']
  return retriever_output


def get_reply(results):
  response = results.get('conditional_router') or results.get('fallback_llm')
  print('from conditional router:', results.get('conditional_router') is not None)
  print('from fallback llm:', results.get('fallback_llm') is not None)
  reply = response['replies'][0].text.replace('\n', '')
  return reply


In [10]:
messages = [
    """مرحباً كيف حالك؟""",
    """أريد أن أعلم ما هو النظام المتبع في المشروع؟"""
]

one_message = []
results = run(messages, HISTORY_LENGTH)
print(get_reply(results))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

from conditional router: True
from fallback llm: False
نظام التقييم يعتمد على ستة مقاييس لتقييم الأداء، بما في ذلك منهجية LLM-as-Judge، حيث يقوم نموذج لغوي كبير بتقييم النظام. يستخدم نظام RAG مع مراحل الفهرسة والإجابة. تُستخدم لغة بايثون و Snowflake لعمليات الحفظ والاسترجاع، مع إطار عمل Haystack-ais لفهم وتخصيص محادثة التفاعل. كان الهدف من المشروع هو تصميم ووكلب محادثة افتراضي يراعي خصوصية المعلومات ويتيح التفاعل مع مصادر محددة مسبقاً.
